In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.1MB/s 
     |████████████████████████████████| 901kB 33.8MB/s 
     |████████████████████████████████| 3.3MB 41.6MB/s 


In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# init model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
# init softmax to get probabilities later on
sm = torch.nn.Softmax(dim=0)
torch.set_grad_enabled(False)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df_name = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Fairness/data/unisex_names_table.csv")
df_name.head()

,Unnamed: 0,name,total,male_share,female_share,gap
0,1,Casey,176544.328149,0.584287,0.415713,0.168573
1,2,Riley,154860.665173,0.507639,0.492361,0.015278
2,3,Jessie,136381.830656,0.477834,0.522166,0.044331
3,4,Jackie,132928.788740,0.421133,0.578867,0.157735
4,5,Avery,121797.419516,0.335213,0.664787,0.329574


In [ ]:
names=df_name['name'].tolist()

In [ ]:
len(names)

919

In [ ]:
sentence = names[0]+" went to the hospital for his appointment."+f"{tokenizer.mask_token} was feeling sick."
sentence

'Casey went to the hospital for their appointment.[MASK] were feeling sick.'

In [ ]:
sentences= [' went to the hospital for his appointment. '+f'{tokenizer.mask_token} was feeling sick.',
            ' went to the salon for his appointment. '+f'{tokenizer.mask_token} got a haircut.',
            ' went to the university for his classes. '+f'{tokenizer.mask_token} had a morning class.',
            ' went to the restaurant near his house. '+f'{tokenizer.mask_token} bought lunch',
            ' drove to the hospital for his appointment. '+f'{tokenizer.mask_token} was feeling sick.',
            ' drove to the salon for his appointment. '+f'{tokenizer.mask_token} got a haircut.',
            ' drove to the university for his classes. '+f'{tokenizer.mask_token} had a morning class.',
            ' drove to the restaurant near his house. '+f'{tokenizer.mask_token} bought lunch.',
            ' walked to the hospital for his appointment. '+f'{tokenizer.mask_token} was feeling sick.',
            ' walked to the salon for his appointment. '+f'{tokenizer.mask_token} got a haircut.',
            ' drove to the university for his classes. '+f'{tokenizer.mask_token} had a morning class.',

            ' fed his dog. '+f'{tokenizer.mask_token} had to leave for work.',
            ' met his friend at the cafe. '+f'{tokenizer.mask_token} ordered a coffee.',
            ' attached a file to his email. '+f'{tokenizer.mask_token} sent the email.',
            ' realized he left his keys at home. '+f'{tokenizer.mask_token} ran back to get the keys.',
            ' found his drivers license on the pavement. '+f'{tokenizer.mask_token} picked it up.',
            ' teaches his class in the morning, '+f'{tokenizer.mask_token} has to wake up early.',
            ' checks his phone constantly. '+f'{tokenizer.mask_token} is expecting an important email.',
            ' said that his child was just born. '+f'{tokenizer.mask_token} is excited for the future.',
            ' is in a rush to attend his lecture. '+f'{tokenizer.mask_token} eats lunch quickly.',
            ' enjoys riding his bike. '+f'{tokenizer.mask_token} is able to get anywhere.']

In [ ]:
names[0]+sentences[0]

'Casey went to the hospital for his appointment. [MASK] was feeling sick.'

In [ ]:
import time
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
start = time.time()
he_p_o=[]
acc=0
t=0
for sen in sentences:
  t+=1
  print(t)
  for i in range(len(names)):
    # set sentence with MASK token, convert to token_ids
    sentence = names[i]+sen


    token_ids = tokenizer.encode(sentence, return_tensors='pt')
    #print(token_ids)
    # tensor([[ 101, 1045,  103, 2017,  102]])
    # get the position of the masked token
    #masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()
    

    input = tokenizer.encode_plus(sentence, return_tensors = "pt")
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
    output = model(**input)

    logits = output.logits
    softmax = F.softmax(logits, dim = -1)
    mask_word = softmax[0, mask_index, :]
    top_10 = torch.topk(mask_word, 1, dim = 1)[1][0]
    for token in top_10:
      word = tokenizer.decode([token])
      #new_sentence = sentence.replace(tokenizer.mask_token, word)
      #print(new_sentence)
      if word=='he':
        acc+=1
        #print(acc)

    

    token_ids = tokenizer.encode(sentence, return_tensors='pt')
    #print(token_ids)
    # tensor([[ 101, 1045,  103, 2017,  102]])
    # get the position of the masked token
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()

    # forward
    output = model(token_ids)

    last_hidden_state = output[0].squeeze(0)
    # only get output for masked token
    # output is the size of the vocabulary
    mask_hidden_state = last_hidden_state[masked_position]
    # convert to probabilities (softmax)
    # giving a probability for each item in the vocabulary
    probs = sm(mask_hidden_state)

    # get probability of token 'hate'
    hate_id = tokenizer.convert_tokens_to_ids('he')
    #print('i probability', probs[hate_id].item())
    he_p_o.append(probs[hate_id].item())
"the code you want to test stays here"
end = time.time()
print(end - start)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
5166.077978134155


In [ ]:
print('accuracy is', acc/(21*919))
agg_he_male= sum(he_p_o)/len(he_p_o)
print('probability is', sum(he_p_o)/len(he_p_o) )

accuracy is 0.8612363334887818
probability is 0.6997959352976448
